In [1]:
!pip install rake_nltk

     |████████████████████████████████| 1.5 MB 3.6 MB/s 
     |████████████████████████████████| 748 kB 38.0 MB/s 
  Attempting uninstall: regex
    Found existing installation: regex 2019.12.20
    Uninstalling regex-2019.12.20:
      Successfully uninstalled regex-2019.12.20
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [2]:
import pandas as pd
from rake_nltk import Rake
import nltk
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

pd.set_option('display.max_columns',100)
df = pd.read_csv("/content/Copy of Chatbot Data Extraction - Symptomate dataset.csv",header=2)
df.head()

,Unnamed: 0,S.No.,Disease Name,Symptoms,Prescription
0,NaN,1,Hemoptysis,High output of blood-streaking of sputum or fr...,"Dextromethorphan, Codeine syrup, Nebulized adr..."
1,NaN,2,Acute rhinitis (Common cold),"Sneezing, nasal congestion with rhinorrhea, mi...","Diphenhydramine, Chlorpheniramine, Cetrizine, ..."
2,NaN,3,Acute bronchitis,"Persistant cough, mild fever and chills, malai...","Dextromethorphan, Levodropropizine, Albuterol,..."
3,NaN,4,Bronchial asthma,Intermittent and reversible attacks of dyspnea...,"Albuterol, Terbutaline, Fluticasone, Budesonid..."
4,NaN,5,Chronic obstructive pulmonary disease (COPD),"Shortness of breath; Frequent coughing, with o...","Albuterol, Levalbuterol, Glycopyrronium, Iprat..."


In [3]:
df.shape

(799, 5)

In [ ]:
df['Disease Name'].unique()

In [5]:
df = df[['Disease Name','Symptoms']]

In [6]:
df.head()

,Disease Name,Symptoms
0,Hemoptysis,High output of blood-streaking of sputum or fr...
1,Acute rhinitis (Common cold),"Sneezing, nasal congestion with rhinorrhea, mi..."
2,Acute bronchitis,"Persistant cough, mild fever and chills, malai..."
3,Bronchial asthma,Intermittent and reversible attacks of dyspnea...
4,Chronic obstructive pulmonary disease (COPD),"Shortness of breath; Frequent coughing, with o..."


In [7]:
df.set_index('Disease Name',inplace=True)
df.head()

,Symptoms
Disease Name,
Hemoptysis,High output of blood-streaking of sputum or fr...
Acute rhinitis (Common cold),"Sneezing, nasal congestion with rhinorrhea, mi..."
Acute bronchitis,"Persistant cough, mild fever and chills, malai..."
Bronchial asthma,Intermittent and reversible attacks of dyspnea...
Chronic obstructive pulmonary disease (COPD),"Shortness of breath; Frequent coughing, with o..."


In [8]:
df = df.dropna()
type(df['Symptoms'][0])

str

In [9]:
df['Symptoms'][0]

'High output of blood-streaking of sputum or frank hemoptysis, anxiety, tachycardia, hypotension, and cold peripheries, fever, night sweat, unexpected weight loss, chest pains'

Remove punctuations

In [10]:
spec_chars = ["!",'"',"#","%","&","'","(",")",
              "*","+",",","-",".","/",":",";","<",
              "=",">","?","@","[","\\","]","^","_",
              "`","{","|","}","~","–"]
for char in spec_chars:
  df['Symptoms'] = df['Symptoms'].str.replace(char, ' ')

df['Symptoms'][0]  

'High output of blood streaking of sputum or frank hemoptysis  anxiety  tachycardia  hypotension  and cold peripheries  fever  night sweat  unexpected weight loss  chest pains'

Tokenise patient comments

In [11]:
from nltk.tokenize import word_tokenize,sent_tokenize

nltk.download('punkt')
# df['Symptoms'] = df['Symptoms'].apply(lambda x: word_tokenize(x))
df = df.apply(lambda row: word_tokenize(row['Symptoms']), axis=1)
df = pd.DataFrame(df,columns=['Symptoms'])
df.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


,Symptoms
Disease Name,
Hemoptysis,"[High, output, of, blood, streaking, of, sputu..."
Acute rhinitis (Common cold),"[Sneezing, nasal, congestion, with, rhinorrhea..."
Acute bronchitis,"[Persistant, cough, mild, fever, and, chills, ..."
Bronchial asthma,"[Intermittent, and, reversible, attacks, of, d..."
Chronic obstructive pulmonary disease (COPD),"[Shortness, of, breath, Frequent, coughing, wi..."


Remove stopwords from patient comments

In [12]:
# Import stopwords with nltk.
from nltk.corpus import stopwords
nltk.download('stopwords')
stop = stopwords.words('english')

# Exclude stopwords with Python's list comprehension and pandas.DataFrame.apply.
df['Symptoms'] = df['Symptoms'].apply(lambda x: ' '.join([word for word in x if word not in (stop)]))
df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,Symptoms
Disease Name,
Hemoptysis,High output blood streaking sputum frank hemop...
Acute rhinitis (Common cold),Sneezing nasal congestion rhinorrhea mild mala...
Acute bronchitis,Persistant cough mild fever chills malaise dry...
Bronchial asthma,Intermittent reversible attacks dyspnea cough ...
Chronic obstructive pulmonary disease (COPD),Shortness breath Frequent coughing without spu...


In [13]:
df['Symptoms'][0]

'High output blood streaking sputum frank hemoptysis anxiety tachycardia hypotension cold peripheries fever night sweat unexpected weight loss chest pains'

In [14]:
from nltk.tokenize import word_tokenize,sent_tokenize

# df['Symptoms'] = df['Symptoms'].apply(lambda x: word_tokenize(x))
df = df.apply(lambda row: word_tokenize(row['Symptoms']), axis=1)
df = pd.DataFrame(df,columns=['Symptoms'])
df['Symptoms'][0]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['High',
 'output',
 'blood',
 'streaking',
 'sputum',
 'frank',
 'hemoptysis',
 'anxiety',
 'tachycardia',
 'hypotension',
 'cold',
 'peripheries',
 'fever',
 'night',
 'sweat',
 'unexpected',
 'weight',
 'loss',
 'chest',
 'pains']

In [23]:
df.shape[0]

458

In [29]:
comment = ['High','output','blood','streaking','sputum','frank','hemoptysis','anxiety','tachycardia','hypotension','cold','peripheries','fever',
           'night','sweat','unexpected','weight','loss','chest','pains']
def tests(comment):
  diseases = []
  for i in range(df.shape[0]):
    res = len(set(df['Symptoms'][i]) & set(comment)) / float(len(set(df['Symptoms'][i]) | set(comment))) * 100
    if res >= 90:
      diseases.append(df.index[i])
  return diseases


In [30]:
diseases = tests(comment)
diseases

['Hemoptysis']

In [41]:
# function for preprocess patient comment
def test_disease(comment):
  # Remove special characters
  spec_chars = ["!",'"',"#","%","&","'","(",")",
              "*","+",",","-",".","/",":",";","<",
              "=",">","?","@","[","\\","]","^","_",
              "`","{","|","}","~","–"]
  for char in spec_chars:
    comment = comment.replace(char, ' ')
 
  #Tokenize string 
  comment = word_tokenize(comment)

  #remove stop words
  stop = stopwords.words('english')
  words  = [word for word in comment if word.lower() not in stop]
  comment = ' '.join(words)
  comment

  #tokenize
  comment = word_tokenize(comment)


  #Find diseases
  diseases = []
  for i in range(df.shape[0]):
    res = len(set(df['Symptoms'][i]) & set(comment)) / float(len(set(df['Symptoms'][i]) | set(comment))) * 100
    if res >= 50:
      diseases.append(df.index[i])

  return diseases


In [42]:
#User input
comment = "high output blood streaking sputum frank hemopytis anxiety tachycardia hypotesion is cold for fever night sweat pains"
if test_disease != 0:
  print("You might be suffering from ",test_disease(comment))

You might be suffering from  ['Hemoptysis']
